In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
#os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/kaggle"

In [3]:
#changing the working directory
#%cd /content/gdrive/My Drive/kaggle

In [4]:
#!kaggle datasets download -d mlg-ulb/creditcardfraud

In [5]:
#unzipping the zip files and deleting the zip files
#!unzip \*.zip  && rm *.zip

In [6]:
!pwd

/content


In [7]:
#Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [8]:
credit = pd.read_csv('/content/gdrive/My Drive/kaggle/creditcard.csv')
credit.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [9]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [10]:
credit.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [11]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [12]:
# selecting the 19 most important features according to our EDA 
X = credit[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 
        'V12', 'V13', 'V14', 'V16', 'V17', 'V21', 'V23', 'V27']]

# scaling the features
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)

y = credit['Class'] # selecting the target variable

In [13]:
# computing the class weight

val_count = credit['Class'].value_counts()
weights = dict(1 / val_count)
weights

{0: 3.51722561243691e-06, 1: 0.0020325203252032522}

In [14]:
# adminstrative cost
admin_cost = 2.5

In [15]:
# defining a function to calculate cost savings
def cost_saving(ytrue, ypred, amount):
    fp = np.sum((ytrue == 0) & (ypred == 1))
    cost = np.sum(fp*admin_cost) + np.sum((amount[(ytrue == 1) & (ypred == 0)]))
    max_cost = np.sum((amount[(ytrue == 1)]))
    savings = 1 - (cost/max_cost)
    
    return savings

In [16]:
cv = StratifiedKFold(n_splits=4, random_state=1, shuffle=True)

In [17]:
# defining a function to calculate cost saving per fold (splits) of our cv
def cost_saving_per_split(scores, x, y, cv_object):
    results = []
    for i, (_, test_ind) in zip(range(cv_object.n_splits), cv_object.split(x, y)):
        ypred = scores['estimator'][i].predict(x[test_ind])
        ytrue = y[test_ind]
        amount = credit['Amount'].values[test_ind]
        results.append(cost_saving(ytrue, ypred, amount))
        
    return results

In [18]:
# defining a function to return a dataframe of metrics results for each fold in our cv
def get_metric_scores(scores, x, y=y, cv_object=cv):
    ind = ['split_'+str(n) for n in range(1, cv_object.n_splits+1)]
    
    scores_credit = pd.DataFrame(index=ind)
    
    scores_credit['f1_score'] = scores['test_f1']
    scores_credit['auc_pr'] = scores['test_average_precision']
    scores_credit['cost_savings'] = cost_saving_per_split(scores, x, y, cv_object)

    return scores_credit

**Fraud Sensitive model (Not cost sensitive)**

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

#GradientBoosting = GradientBoostingClassifier()

In [20]:
fraud_sensitive_model = GradientBoostingClassifier()
fraud_sensitive_scores = cross_validate(fraud_sensitive_model, scaled_X, y, 
                                        scoring=['f1', 'average_precision'], 
                                        cv=cv, n_jobs=4, return_estimator=True)

In [21]:
fraud_sensitive_results = get_metric_scores(fraud_sensitive_scores, scaled_X)
fraud_sensitive_results

,f1_score,auc_pr,cost_savings
split_1,0.463277,0.371886,0.495092
split_2,0.660194,0.418952,0.537342
split_3,0.763636,0.626318,0.678207
split_4,0.697674,0.514382,0.592574


In [22]:
fraud_sensitive_results.mean()

f1_score        0.646195
auc_pr          0.482884
cost_savings    0.575804
dtype: float64

**Cost Sensitive model**

In [23]:
sample_weights = np.array([credit['Amount'][ind] if fraud else admin_cost for ind, fraud in enumerate(y)])

In [24]:
cost_sensitive_model = GradientBoostingClassifier()
cost_sensitive_scores = cross_validate(cost_sensitive_model, scaled_X, y, 
                                       scoring=['f1', 'average_precision'], 
                                       cv=cv, n_jobs=4, return_estimator=True, 
                                       fit_params={'sample_weight': sample_weights})

In [25]:
cost_sensitive_results = get_metric_scores(cost_sensitive_scores, scaled_X)
cost_sensitive_results

,f1_score,auc_pr,cost_savings
split_1,0.739300,0.684631,0.723344
split_2,0.785714,0.754631,0.751873
split_3,0.803279,0.754557,0.813855
split_4,0.804979,0.746693,0.735220


In [26]:
cost_sensitive_results.mean()

f1_score        0.783318
auc_pr          0.735128
cost_savings    0.756073
dtype: float64

**Cost Sensitive Model - SMOTE**

In [34]:

smote = SMOTE(random_state=1)
cost_sensitive_smote = GradientBoostingClassifier()

pipe_smote = Pipeline([('smote', smote), ('model', cost_sensitive_smote)])

smote_scores = cross_validate(pipe_smote, scaled_X, y, scoring=['f1', 'average_precision'], 
                              cv=cv, n_jobs=4, return_estimator=True, 
                              fit_params={'sample_weight': sample_weights})


In [35]:
smote_results = get_metric_scores(smote_scores, scaled_X)
smote_results

,f1_score,auc_pr,cost_savings
split_1,0.962458,0.994608,-3.320568
split_2,0.961224,0.994380,-7.322638
split_3,0.961987,0.994470,-5.079398
split_4,0.961487,0.994560,-4.944670


In [36]:
smote_results.mean()

f1_score        0.961789
auc_pr          0.994504
cost_savings   -5.166818
dtype: float64

**Cost sensitive - Undersampling**

In [42]:
from imblearn.under_sampling import RandomUnderSampler

undersample = RandomUnderSampler()

cost_sensitive_under = GradientBoostingClassifier()

pipe_under = Pipeline([('undersampler', undersample), ('model', cost_sensitive_under)])

undersample_scores = cross_validate(pipe_under, scaled_X, y, scoring=['f1', 'average_precision'], 
                                    cv=cv, n_jobs=4, return_estimator=True, 
                                    fit_params={'sample_weight': sample_weights})


In [43]:
undersample_results = get_metric_scores(undersample_scores, scaled_X)
undersample_results

,f1_score,auc_pr,cost_savings
split_1,0.951613,0.988808,-4.883160
split_2,0.911290,0.971638,-8.014542
split_3,0.875969,0.965640,-5.271748
split_4,0.932806,0.980312,-6.609302


In [44]:
undersample_results.mean()

f1_score        0.917920
auc_pr          0.976600
cost_savings   -6.194688
dtype: float64